In [1]:
#importando as bibliotecas
from pyspark import SparkContext
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pandas as pd
import csv


#localização dos arquivos no Databricks
data_a = "/FileStore/tables/NASA_Jul95.csv"
data_b = "/FileStore/tables/NASA_Aug95.csv"

In [2]:
#Iniciando a sessão Spark usando SparkSession
spark = SparkSession \
    .builder \
    .appName("Python Spark - Nasa") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [3]:
#Lendo CSV data_a data_b
df_data_a = spark.read.format("csv")\
            .option("delimiter", " ")\
            .option("multilines", "true")\
            .option("quotes", '""')\
            .option("header", "false")\
            .load(data_a)\
            .toDF("host", "a", "b", "dias", "time_zone", "request", "http", "bytes")

df_data_b = spark.read.format("csv")\
            .option("delimiter", " ")\
            .option("multilines", "true")\
            .option("quotes", '""')\
            .option("header", "false")\
            .load(data_b)\
            .toDF("host", "a", "b", "dias", "time_zone", "request", "http", "bytes")

In [4]:
#mostrando os dez primeiros resultados de NASA_Jul95
df_data_a.show(10,truncate = False)

+--------------------+---+---+---------------------+---------+------------------------------------------------------------+----+-----+
host |a |b |dias |time_zone|request |http|bytes|
+--------------------+---+---+---------------------+---------+------------------------------------------------------------+----+-----+
199.72.81.55 |- |- |[01/Jul/1995:00:00:01|-0400] |GET /history/apollo/ HTTP/1.0 |200 |6245 |
unicomp6.unicomp.net|- |- |[01/Jul/1995:00:00:06|-0400] |GET /shuttle/countdown/ HTTP/1.0 |200 |3985 |
199.120.110.21 |- |- |[01/Jul/1995:00:00:09|-0400] |GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0 |200 |4085 |
burger.letters.com |- |- |[01/Jul/1995:00:00:11|-0400] |GET /shuttle/countdown/liftoff.html HTTP/1.0 |304 |0 |
199.120.110.21 |- |- |[01/Jul/1995:00:00:11|-0400] |GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0|200 |4179 |
burger.letters.com |- |- |[01/Jul/1995:00:00:12|-0400] |GET /images/NASA-logosmall.gif HTTP/1.0 |304 |0 |
burger.letters.com |- |- |[01/Jul/1995:00:00:12|-0400] |GET /shuttle/countdown/video/livevideo.gif HTTP/1.0 |200 |0 |
205.212.115.106 |- |- |[01/Jul/1995:00:00:12|-0400] |GET /shuttle/countdown/countdown.html HTTP/1.0 |200 |3985 |
d104.aa.net |- |- |[01/Jul/1995:00:00:13|-0400] |GET /shuttle/countdown/ HTTP/1.0 |200 |3985 |
129.94.144.152 |- |- |[01/Jul/1995:00:00:13|-0400] |GET / HTTP/1.0 |200 |7074 |
+--------------------+---+---+---------------------+---------+------------------------------------------------------------+----+-----+
only showing top 10 rows

In [5]:
#Vamos tirar algumas colunas desnecessárias em nosso data Frame utilizando spark.sql
df_data_a.createOrReplaceTempView("NASA_Jul95")
df_data_b.createOrReplaceTempView("NASA_Aug95")

df1 = spark.sql("Select host, dias, request, http, bytes from NASA_Jul95")
df2 = spark.sql("Select host, dias, request, http, bytes from NASA_Aug95")

#Vamos fazer um union dos dois datasets para respondermos as perguntas abaixo 
access_log = df1.union(df2)

In [6]:
#Vamos utlizar a função select com filtro em host e distinct

access_log.select("host").distinct().count()

Out[6]: 137979

In [7]:
access_log.select('host').distinct().count()




Out[7]: 137979

In [8]:
#Existem algumas maneiras de fazermos esse filtro pelo código de erro 404, primeiro vamos usar código SQL
access_log.createOrReplaceTempView("Erro404")

spark.sql("Select * from Erro404 where http='404'").count()

Out[8]: 20871

In [9]:
#Segundo vamos usar filter
access_log.filter(access_log['http'].contains('404')).count()

Out[9]: 20871

In [10]:
#Primeiro exemplo vamos usar SQL para mostrar as 5 URLs com mais erros
spark.sql("Select host, count(host) as qtde from Erro404 where http='404' group by host order by 2 desc limit 5").show(10, truncate=False)

+---------------------------+----+
host |qtde|
+---------------------------+----+
hoohoo.ncsa.uiuc.edu |251 |
piweba3y.prodigy.com |156 |
jbiagioni.npt.nuwc.navy.mil|132 |
piweba1y.prodigy.com |114 |
www-d4.proxy.aol.com |91 |
+---------------------------+----+

In [11]:
#Segundo vamos utilizar funções
access_log.select('host').filter(access_log['http'].contains('404')).groupBy('host').count().orderBy('count',ascending=False).limit(5).show(truncate=False)

+---------------------------+-----+
host |count|
+---------------------------+-----+
hoohoo.ncsa.uiuc.edu |251 |
piweba3y.prodigy.com |156 |
jbiagioni.npt.nuwc.navy.mil|132 |
piweba1y.prodigy.com |114 |
www-d4.proxy.aol.com |91 |
+---------------------------+-----+

In [12]:
#Quantidade de erros por dia

def get_dia(timestamp):
    data = timestamp.split('/')
    return data[0][1:] + "/" + data[1] + "/" + data[2][:4]

erros = access_log.filter(access_log['http'].contains('404'))

udf_custom = F.udf(get_dia)
dias = erros.withColumn("Erros_dias", udf_custom(erros["dias"]))

dias.select("Erros_dias").groupBy("Erros_dias").count().orderBy("count", ascending=False)

dias.show()

+--------------------+--------------------+--------------------+----+-----+-----------+
 host| dias| request|http|bytes| Erros_dias|
+--------------------+--------------------+--------------------+----+-----+-----------+
dd15-062.compuser...|[01/Jul/1995:00:0...|GET /news/sci.spa...| 404| -|01/Jul/1995|
 netport-27.iu.net|[01/Jul/1995:00:1...|GET /pub/winvn/re...| 404| -|01/Jul/1995|
 netport-27.iu.net|[01/Jul/1995:00:1...|GET /pub/winvn/re...| 404| -|01/Jul/1995|
blv-pm0-ip28.halc...|[01/Jul/1995:00:1...|GET /persons/astr...| 404| -|01/Jul/1995|
blv-pm0-ip28.halc...|[01/Jul/1995:00:1...|GET /persons/astr...| 404| -|01/Jul/1995|
cu-dialup-1005.ci...|[01/Jul/1995:00:1...|GET /pub/winvn/re...| 404| -|01/Jul/1995|
cu-dialup-1005.ci...|[01/Jul/1995:00:1...|GET /pub/winvn/re...| 404| -|01/Jul/1995|
cu-dialup-1005.ci...|[01/Jul/1995:00:1...|GET /pub/winvn/re...| 404| -|01/Jul/1995|
 mimas.execpc.com|[01/Jul/1995:00:1...|GET /shuttle/miss...| 404| -|01/Jul/1995|
zoom112.telepath.com|[01/Jul/1995:00:2...|GET /history/apol...| 404| -|01/Jul/1995|
ad08-027.compuser...|[01/Jul/1995:00:3...|GET /shuttle/miss...| 404| -|01/Jul/1995|
ad08-027.compuser...|[01/Jul/1995:00:3...|GET /shuttle/miss...| 404| -|01/Jul/1995|
piweba3y.prodigy.com|[01/Jul/1995:00:4...|GET /shuttle/miss...| 404| -|01/Jul/1995|
piweba3y.prodigy.com|[01/Jul/1995:00:4...|GET /shuttle/miss...| 404| -|01/Jul/1995|
 204.212.153.234|[01/Jul/1995:00:4...|GET /pub/winvn/re...| 404| -|01/Jul/1995|
 204.212.153.234|[01/Jul/1995:00:4...|GET /pub/winvn/re...| 404| -|01/Jul/1995|
ix-wbg-va2-26.ix....|[01/Jul/1995:00:4...|GET /history/apol...| 404| -|01/Jul/1995|
ix-wbg-va2-26.ix....|[01/Jul/1995:00:4...|GET /history/apol...| 404| -|01/Jul/1995|
ix-wbg-va2-26.ix....|[01/Jul/1995:00:4...|GET /history/apol...| 404| -|01/Jul/1995|
ix-wbg-va2-26.ix....|[01/Jul/1995:00:4...|GET /history/apol...| 404| -|01/Jul/1995|
+--------------------+--------------------+--------------------+----+-----+-----------+
only showing top 20 rows

In [13]:
spark.sql("Select substring(dias, 2,11) as dias from Erro404 where http='404'").show(10, truncate=False)


+-----------+
dias |
+-----------+
01/Jul/1995|
01/Jul/1995|
01/Jul/1995|
01/Jul/1995|
01/Jul/1995|
01/Jul/1995|
01/Jul/1995|
01/Jul/1995|
01/Jul/1995|
01/Jul/1995|
+-----------+
only showing top 10 rows

In [14]:
access_log.select(F.sum('bytes')).show()

+---------------+
 sum(bytes)|
+---------------+
6.5524319796E10|
+---------------+

In [15]:
access_log.createOrReplaceTempView("TotalBytes")

spark.sql("Select sum(bytes) as total_bytes from TotalBytes").show(truncate=False)

+---------------+
total_bytes |
+---------------+
6.5524319796E10|
+---------------+